In [1]:
# imports
import importlib as imp
import geopandas as gpd
import pandas as pd
import functions as f
# debug
imp.reload(f)

<module 'functions' from 'd:\\ITU\\Geospatial Data Science\\PROJECT\\functions.py'>

### Bikeliane preprocessing

In [3]:
bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_crs_25832.parquet')

In [26]:
test = bikelane_all.sample(100000)

In [11]:
t = f.buffer_and_union_and_simplify_geopandas(test,
                                    buffer=50,
                                    simplify_value=1,
                                    debug=True)

buffer objects start
unary union starts
simplify objects start


### Here you can prepare the data

Takes forever, there are too many objects in the db. Instead I try to slice it

In [13]:
# dont run it at all

METERS = [50,100]

for m in METERS:
    t = f.buffer_and_union_and_simplify_geopandas(bikelane_all,
                                    buffer=m,
                                    simplify_value=1,
                                    debug=True)
    
    t.to_parquet(f'dataset/processed/bikelane_buffered_{m}_simplify_1_DENMAKR_CRS.parquet')


buffer objects start
unary union starts


: 

: 

In [46]:
SLICE = 250
slices = len(bikelane_all) / SLICE
total_elements = len(bikelane_all)

In [47]:
slices

13031.66

In [49]:
BUFFER = 50
for sl in range(0, SLICE):
    start = sl * total_elements/SLICE
    end = (sl + 1) * total_elements/SLICE
    t = f.buffer_and_union_and_simplify_geopandas(bikelane_all[int(start):int(end)],
                                    buffer=BUFFER,
                                    simplify_value=1,
                                    debug=True)
    
    t.to_parquet(f'dataset/processed/bikelane_buffered_{BUFFER}_simplify_1_DENMAKR_CRS_{int(start)}_{int(end)}.parquet')

buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts
unary union ends
simplify objects start
buffer objects start
unary union starts


In [37]:
# read in the files and make an unary union
gpd_df = gpd.GeoDataFrame()
for sl in range(0, SLICE):
    start = sl * total_elements/SLICE
    end = (sl + 1) * total_elements/SLICE
    t = gpd.read_parquet(f'dataset/processed/bikelane_buffered_{BUFFER}_simplify_1_DENMAKR_CRS_{int(start)}_{int(end)}.parquet')
    gpd_df = gpd.GeoDataFrame(pd.concat([gpd_df, t], ignore_index=True))
    

In [39]:
ttt = f.buffer_and_union_and_simplify_geopandas(gpd_df,buffer=0, simplify_value=0, debug=True)

buffer objects start
unary union starts
unary union ends


### Bikeline to multistring, conversion, etc etc etc .... BORING

bikelane_all = gpd.read_parquet('dataset/raw_unprocessed/bikelane_dk_crs_25832.parquet')

In [ ]:
# Concatenate all LineString geometries into a single MultiLineString
all_line_geoms = MultiLineString(bikelane_all['geometry'].tolist())

# Create a GeoDataFrame with the MultiLineString geometry
multi_line_gdf = gpd.GeoDataFrame(geometry=[all_line_geoms])

# Print the GeoDataFrame
print(multi_line_gdf)

In [ ]:
multi_line_gdf.to_parquet('dataset/raw_unprocessed/bikelane_dk_crs_25832_multilinestring.parquet')
multi_line_gdf.crs = f.DENMARK_CRS
multi_line_gdf_wgs84 = multi_line_gdf.to_crs(epsg=4326)
multi_line_gdf_wgs84.to_parquet('dataset/raw_unprocessed/bikelane_wgs84_multilinestring.parquet')